In [ ]:
using Distributed
addprocs(40)
include("./fermionise.jl")
include("./models.jl")
include("./quantuminfo.jl")

In [ ]:
function main(numBath)
    basisStates = BasisStates(2 * (1 + numBath))
    hamiltonian = TOSHamiltonianRSpace(basisStates, div(numBath, 2), [1, 20, 1, 10, 1])
    E, X = diagonalise(basisStates, hamiltonian)
    println(E)
    I2_dA = MutualInfo(X[2], [1; 2], [3; 4])
    I2_dB = MutualInfo(X[2], [1; 2], [6; 5])
    return I2_dA, I2_dB
end

main(4)